In [1]:
import pandas as pd
from sqlalchemy import create_engine
import yaml


Crear el DataFrame con cada hora del día (0 a 23)


In [2]:
fechas = pd.date_range(start="2023-01-01 00:00:00", end="2023-12-31 23:00:00", freq="H")

# Crear el DataFrame Dim_Tiempo
dim_tiempo = pd.DataFrame({
    "Año": fechas.year,
    "Mes": fechas.month,
    "Dia": fechas.day,
    "Hora": fechas.hour
})

dim_tiempo

C:\Users\Eyder\AppData\Local\Temp\ipykernel_32584\4087629111.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  fechas = pd.date_range(start="2023-01-01 00:00:00", end="2023-12-31 23:00:00", freq="H")


,Año,Mes,Dia,Hora
0,2023,1,1,0
1,2023,1,1,1
2,2023,1,1,2
3,2023,1,1,3
4,2023,1,1,4
...,...,...,...,...
8755,2023,12,31,19
8756,2023,12,31,20
8757,2023,12,31,21
8758,2023,12,31,22


Añadir una columna para el ID único de cada hora


In [3]:
dim_tiempo["tiempo_key"] = dim_tiempo.index


Reorganizar columnas para que Tiempo_id sea la primera


In [4]:
dim_tiempo = dim_tiempo[["tiempo_key", "Año", "Mes", "Dia", "Hora"]]

Cargar configuración de la base de datos desde el archivo YAML


In [5]:
with open('../../configBD/config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_etl = config['bodega']

Construir la URL de conexión a la base de datos


In [6]:
url_etl = (f"{config_etl['driver']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['db']}")

Crear el motor de conexión a la base de datos


In [7]:
cliente_etl = create_engine(url_etl)

Guardar el DataFrame en la base de datos como la tabla dim_tiempo


In [8]:
dim_tiempo.to_sql('dim_tiempo', cliente_etl, if_exists='replace', index=False, index_label='Tiempo_id')


760

In [9]:
dim_tiempo

,tiempo_key,Año,Mes,Dia,Hora
0,0,2023,1,1,0
1,1,2023,1,1,1
2,2,2023,1,1,2
3,3,2023,1,1,3
4,4,2023,1,1,4
...,...,...,...,...,...
8755,8755,2023,12,31,19
8756,8756,2023,12,31,20
8757,8757,2023,12,31,21
8758,8758,2023,12,31,22
